# Analysis of BXD 

## Preparatory steps

In [1]:
# load packages needed
# install.packages("lme4")
# install.packages("tidyverse")
# install.packages("vioplot")
# install.packages("metafor")
library(tidyverse)
library(vioplot)
library(lme4)
library(metafor)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.1
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: sm

Package 'sm', version 2.2-5.6: type help(sm) for summary information

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading 'metafor' package (version 2.4-0). For an overview 
and introduction to the package please type: help(metafor).



In [2]:
BXDagingbwt <- "../data/AgingBXDNOV2018dataSR2.csv"
agingbxd <- read.csv(BXDagingbwt)
names(agingbxd)

[1] "IDFemaleAgingColony"   "EarTagNumberCurrent"   "EarTagNumberOriginal" 
 [4] "StrainNameCurrent"     "DietCode"              "Sex"                  
 [7] "CoatColorOfCase"       "GenerationOfCase"      "CauseOfDeath"         
[10] "AgeAtDeath"            "DateOfBirth"           "DateDietStart"        
[13] "DateOfDeath"           "EntryAgeInColony"      "Weight00Baseline"     
[16] "logEntryAgeInColony"   "AdjWeightBaseline"     "AdjWeightBaseline2"   
[19] "Weight01"              "Weight02"              "WeightGainPost100days"
[22] "Weight03"              "Weight04"              "Weight05"             
[25] "Weight06"              "WeightGainPost400days" "Weight07"             
[28] "Weight08"              "Weight09"              "Weight10"             
[31] "Weight11"              "Weight12"              "Weight13"             
[34] "Weight14"              "DaysOnDietAtWeight01"  "DaysOnDietAtWeight02" 
[37] "DaysOnDietAtWeight03"  "DaysOnDietAtWeight04"  "DaysOnDietAtWeight05" 
[40] "DaysOnDietAtWeight06"  "DaysOnDietAtWeight07"  "DaysOnDietAtWeight08" 
[43] "DaysOnDietAtWeight09"  "DaysOnDietAtWeight10"  "DaysOnDietAtWeight11" 
[46] "DaysOnDietAtWeight12"  "DaysOnDietAtWeight13"  "DaysOnDietAtWeight14" 
[49] "AgeAtWeight01"         "AgeAtWeight02"         "AgeAtWeight03"        
[52] "AgeAtWeight04"         "AgeAtWeight05"         "AgeAtWeight06"        
[55] "AgeAtWeight07"         "AgeAtWeight08"         "AgeAtWeight09"        
[58] "AgeAtWeight10"         "AgeAtWeight11"         "AgeAtWeight12"        
[61] "AgeAtWeight13"         "AgeAtWeight14"         "FearConditioned"

In [3]:
summarizeByDiet <- function(data,pheno,nmin=3)
    {
    strainByDiet <- group_by(data,DietCode,StrainNameCurrent) %>%
          summarize(n=n()) %>%
          pivot_wider(id_cols = "StrainNameCurrent",names_from = "DietCode",values_from = "n")
    strainsWithnmin <- filter(strainByDiet,CD>nmin) %>% filter(HF>nmin) %>% select(StrainNameCurrent)
    newdata <- filter(data,StrainNameCurrent %in% strainsWithnmin$StrainNameCurrent) %>%
             mutate(StrainNameCurrent=fct_drop(fct_explicit_na((StrainNameCurrent))))
    
    # lm
    outLM <- lm(formula(paste(pheno,"~DietCode+StrainNameCurrent")),data=newdata)
    # lmx
    outLMX <- lm(formula(paste(pheno,"~DietCode*StrainNameCurrent")),data=newdata)
    # lmm
    outLMM <- lmer(formula(paste(pheno,"~DietCode+(1+DietCode|StrainNameCurrent)")),data=newdata)
    summary(outLMM)
    # meta analysis
    pheno <- as.name(pheno)
    strainByDiet <- group_by(newdata,DietCode,StrainNameCurrent) %>% 
                summarize(n:=sum(!is.na( {{ pheno }}  )),
                          mean:=mean( {{ pheno }},na.rm=T),
                          sd := sd( {{ pheno }},na.rm=T)) %>%
                mutate(se=sd/sqrt(n)) %>%
                pivot_wider(id_cols = "StrainNameCurrent",
                            names_from = c("DietCode"),values_from = c("n","mean","sd"))

   outMeta <- rma.uni(measure="MD",m1i=mean_HF,m2i=mean_CD,
                      sd1i=sd_HF,sd2i=sd_CD,n1i=n_HF,n2i=n_CD,
                      data=strainByDiet)
    
    lmtab <- as_tibble(coef(summary(outLM)),rownames="coefname") %>% filter(!grepl("Strain",coefname))
    av <- anova(outLM)
    avpct <- round(1000*av[,2]/sum(av[,2]))/10
    names(avpct) <- rownames(av)
    lmxtab <- as_tibble(coef(summary(outLMX)),rownames="coefname") %>%    filter(!grepl("Strain",coefname))
    avx <- anova(outLMX)
    avxpct <- round(1000*avx[,2]/sum(avx[,2]))/10
    names(avxpct) <- rownames(avx)
    lmmtab <- coef(summary(outLMM))
    metatab <- coef(summary(outMeta))
    rownames(metatab) <- "DietCodeHF"
    
    cat("\n --- ANOVA --- \n")
    print(avpct)
    cat("\n --- LM --- \n")
    print(lmtab)
    cat("\n --- ANOVAX --- \n")
    print(avxpct)
    cat("\n --- LMX --- \n")
    print(lmxtab)
    cat("\n --- LMM --- \n")
    print(lmmtab)
    cat("\n --- Meta --- \n")
    print(metatab)

   list(outLM=outLM,outLMX=outLMX,outLMM=outLMM,outMeta=outMeta,strainByDiet=strainByDiet)
}

out <- summarizeByDiet(agingbxd,"AgeAtDeath")

`summarise()` regrouping output by 'DietCode' (override with `.groups` argument)

`summarise()` regrouping output by 'DietCode' (override with `.groups` argument)




 --- ANOVA --- 
         DietCode StrainNameCurrent         Residuals 
              4.7              29.5              65.8 

 --- LM --- 
# A tibble: 2 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)    709.         27.5      25.8   1.67e-116
2 DietCodeHF     -85.7         8.97     -9.56  6.64e- 21

 --- ANOVAX --- 
                  DietCode          StrainNameCurrent 
                       4.7                       29.5 
DietCode:StrainNameCurrent                  Residuals 
                       6.2                       59.6 

 --- LMX --- 
# A tibble: 2 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)     728.         35.9     20.3    3.85e-78
2 DietCodeHF     -129.         53.3     -2.41   1.60e- 2

 --- LMM --- 
            Estimate Std. Error  t value
(Intercept) 694.4516   16.09812 43.13868
DietCodeHF  -86.6

In [4]:
summarizeByCovariate <- function(data,pheno,cov,nmin=3)
    {
    covname <- as.name(cov)

    byStrain <- group_by(data,StrainNameCurrent) %>%
          summarize(n=n())
    strainsWithnmin <- filter(byStrain,n>nmin) %>% select(StrainNameCurrent)
    newdata <- filter(data,StrainNameCurrent %in% strainsWithnmin$StrainNameCurrent) %>%
             mutate(StrainNameCurrent=fct_drop(fct_explicit_na((StrainNameCurrent))))
    
    # lm
    outLM <- lm(formula(paste(pheno,"~",cov)),data=newdata)
    # lmm
    outLMM <- lmer(formula(paste(pheno,"~",cov,"+(1|StrainNameCurrent)")),data=newdata)
    # meta analysis
    covLM <- function(df)
        {
        lm(formula(paste(pheno,"~",cov)),data=df)
        }
    pheno <- as.name(pheno)
    strainByCov <- group_by(newdata,StrainNameCurrent) %>% 
                   nest() %>% 
                   mutate(lmresult=map(data,covLM)) %>%
                   mutate(beta=map(lmresult,broom::tidy)) %>% 
                   unnest(beta) %>%
                   filter(term==cov)
    outMeta <- rma.uni(yi=strainByCov$estimate,sei=strainByCov$std.error)

    lmtab <- as_tibble(coef(summary(outLM)),rownames="coefname") %>% filter(!grepl("Strain",coefname))
    av <- anova(outLM)
    avpct <- round(1000*av[,2]/sum(av[,2]))/10
    names(avpct) <- rownames(av)
    lmmtab <- coef(summary(outLMM))
    metatab <- coef(summary(outMeta))
    rownames(metatab) <- cov
    
    cat("\n --- ANOVA --- \n")
    print(avpct)
    cat("\n --- LM --- \n")
    print(lmtab)
    cat("\n --- LMM --- \n")
    print(lmmtab)
    cat("\n --- Meta --- \n")
    print(metatab)

   
   list(outLM=outLM,outLMM=outLMM,outMeta=outMeta)
}
out <- summarizeByCovariate(agingbxd,"AgeAtDeath","Weight02")

`summarise()` ungrouping output (override with `.groups` argument)

Warning message in rma.uni(yi = strainByCov$estimate, sei = strainByCov$std.error):
“Studies with NAs omitted from model fitting.”



 --- ANOVA --- 
 Weight02 Residuals 
      5.4      94.6 

 --- LM --- 
# A tibble: 2 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)   795.         16.7       47.7   1.95e-284
2 Weight02       -3.93        0.463     -8.49  5.74e- 17

 --- LMM --- 
             Estimate Std. Error   t value
(Intercept) 796.33768 19.4085791 41.030190
Weight02     -3.96807  0.4570734 -8.681472

 --- Meta --- 
          estimate        se     zval         pval     ci.lb     ci.ub
Weight02 -4.419632 0.6226359 -7.09826 1.263371e-12 -5.639976 -3.199288


In [5]:
summarizeByCovariate2 <- function(data,pheno,cov,backCov,nmin=3)
    {
    covname <- as.name(cov)

    byStrain <- group_by(data,StrainNameCurrent) %>%
          summarize(n=n())
    strainsWithnmin <- filter(byStrain,n>nmin) %>% select(StrainNameCurrent)
    newdata <- filter(data,StrainNameCurrent %in% strainsWithnmin$StrainNameCurrent) %>%
             mutate(StrainNameCurrent=fct_drop(fct_explicit_na((StrainNameCurrent))))
    
    # lm
    outLM <- lm(formula(paste(pheno,"~",cov,"+",backCov,"+StrainNameCurrent")),data=newdata)
    # lmm
    outLMM <- lmer(formula(paste(pheno,"~",cov,"+",backCov,"+(1|StrainNameCurrent)")),data=newdata)
    # meta analysis
    covLM <- function(df)
        {
        lm(formula(paste(pheno,"~",cov,"+",backCov)),data=df)
        }
    pheno <- as.name(pheno)
    strainByCov <- group_by(newdata,StrainNameCurrent) %>% 
                   nest() %>% 
                   mutate(lmresult=map(data,covLM)) %>%
                   mutate(beta=map(lmresult,broom::tidy)) %>% 
                   unnest(beta) %>%
                   filter(grepl(cov,term))
    outMeta <- rma.uni(yi=strainByCov$estimate,sei=strainByCov$std.error)
                   

   # outMeta <- rma.uni(measure="MD",m1i=mean_HF,m2i=mean_CD,
   #                   sd1i=sd_HF,sd2i=sd_CD,n1i=n_HF,n2i=n_CD,
   #                   data=strainByDiet)
    lmtab <- as_tibble(coef(summary(outLM)),rownames="coefname") %>% filter(!grepl("Strain",coefname))
    av <- anova(outLM)
    avpct <- round(1000*av[,2]/sum(av[,2]))/10
    names(avpct) <- rownames(av)
    lmmtab <- coef(summary(outLMM))
    metatab <- coef(summary(outMeta))
    rownames(metatab) <- cov
    
    cat("\n --- ANOVA --- \n")
    print(avpct)
    cat("\n --- LM --- \n")
    print(lmtab)
    cat("\n --- LMM --- \n")
    print(lmmtab)
    cat("\n --- Meta --- \n")
    print(metatab)
   list(outLM=outLM,outLMM=outLMM,outMeta=outMeta)
}
out <- summarizeByCovariate2(agingbxd,"AgeAtDeath","Weight06","DietCode",20)


`summarise()` ungrouping output (override with `.groups` argument)




 --- ANOVA --- 
         Weight06          DietCode StrainNameCurrent         Residuals 
              8.6               3.6              22.3              65.5 

 --- LM --- 
# A tibble: 3 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)   831.         29.1       28.6   2.65e-107
2 Weight06       -1.25        0.470     -2.67  7.83e-  3
3 DietCodeHF    -73.0        14.4       -5.05  6.08e-  7

 --- LMM --- 
              Estimate Std. Error   t value
(Intercept) 807.063385  19.423195 41.551526
Weight06     -1.152949   0.457566 -2.519744
DietCodeHF  -73.802621  14.264865 -5.173734

 --- Meta --- 
           estimate        se     zval      pval   ci.lb     ci.ub
Weight06 -0.6686714 0.4617069 -1.44826 0.1475444 -1.5736 0.2362574


## Looking at the effect of diet on longevity across all strains and adjusting for strain effect

In [6]:
out1 <- summarizeByDiet(agingbxd,"AgeAtDeath")
summary(out1$outMeta)

`summarise()` regrouping output by 'DietCode' (override with `.groups` argument)

`summarise()` regrouping output by 'DietCode' (override with `.groups` argument)




 --- ANOVA --- 
         DietCode StrainNameCurrent         Residuals 
              4.7              29.5              65.8 

 --- LM --- 
# A tibble: 2 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)    709.         27.5      25.8   1.67e-116
2 DietCodeHF     -85.7         8.97     -9.56  6.64e- 21

 --- ANOVAX --- 
                  DietCode          StrainNameCurrent 
                       4.7                       29.5 
DietCode:StrainNameCurrent                  Residuals 
                       6.2                       59.6 

 --- LMX --- 
# A tibble: 2 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)     728.         35.9     20.3    3.85e-78
2 DietCodeHF     -129.         53.3     -2.41   1.60e- 2

 --- LMM --- 
            Estimate Std. Error  t value
(Intercept) 694.4516   16.09812 43.13868
DietCodeHF  -86.6


Random-Effects Model (k = 57; tau^2 estimator: REML)

   logLik   deviance        AIC        BIC       AICc 
-340.8002   681.6004   685.6004   689.6511   685.8268   

tau^2 (estimated amount of total heterogeneity): 7127.6115 (SE = 2149.5993)
tau (square root of estimated tau^2 value):      84.4252
I^2 (total heterogeneity / total variability):   65.96%
H^2 (total variability / sampling variability):  2.94

Test for Heterogeneity:
Q(df = 56) = 176.7612, p-val < .0001

Model Results:

estimate       se     zval    pval      ci.lb     ci.ub 
-88.7735  14.2686  -6.2216  <.0001  -116.7394  -60.8075  *** 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Age at entry into colony and start of HFD is not associated with longevity adjusting for strain effect

In [7]:
out2 <- summarizeByCovariate(agingbxd,"AgeAtDeath","EntryAgeInColony")

`summarise()` ungrouping output (override with `.groups` argument)

Warning message in rma.uni(yi = strainByCov$estimate, sei = strainByCov$std.error):
“Studies with NAs omitted from model fitting.”



 --- ANOVA --- 
EntryAgeInColony        Residuals 
             0.5             99.5 

 --- LM --- 
# A tibble: 2 x 5
  coefname         Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>               <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)       627.          9.37       66.9     0      
2 EntryAgeInColony    0.139       0.0532      2.62    0.00881

 --- LMM --- 
                    Estimate  Std. Error  t value
(Intercept)      626.9636317 14.45178287 43.38313
EntryAgeInColony   0.1623257  0.05147999  3.15318

 --- Meta --- 
                  estimate        se     zval      pval       ci.lb     ci.ub
EntryAgeInColony 0.1216479 0.1004456 1.211082 0.2258641 -0.07522197 0.3185177


## Looking at the effect of initial weight at entry into aging colony on longevity adjusting for age at entry into colony

In [8]:
out3 <- summarizeByCovariate2(agingbxd,"AgeAtDeath","Weight00Baseline","EntryAgeInColony",4)
summary(out3$outMeta)

`summarise()` ungrouping output (override with `.groups` argument)




 --- ANOVA --- 
 Weight00Baseline  EntryAgeInColony StrainNameCurrent         Residuals 
              1.3               2.2              26.6              70.0 

 --- LM --- 
# A tibble: 3 x 5
  coefname         Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>               <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)       909.         64.1        14.2    2.30e-42
2 Weight00Baseline   -5.05        1.20       -4.19   2.94e- 5
3 EntryAgeInColony    0.278       0.0586      4.75   2.31e- 6

 --- LMM --- 
                    Estimate  Std. Error   t value
(Intercept)      724.8897892 26.83498304 27.012866
Weight00Baseline  -5.0569494  1.15236191 -4.388334
EntryAgeInColony   0.2818145  0.05751683  4.899688

 --- Meta --- 
                  estimate       se      zval         pval     ci.lb     ci.ub
Weight00Baseline -5.937299 1.732012 -3.427977 0.0006080964 -9.331981 -2.542617



Random-Effects Model (k = 66; tau^2 estimator: REML)

   logLik   deviance        AIC        BIC       AICc 
-288.0194   576.0388   580.0388   584.3875   580.2323   

tau^2 (estimated amount of total heterogeneity): 37.8894 (SE = 26.6341)
tau (square root of estimated tau^2 value):      6.1554
I^2 (total heterogeneity / total variability):   24.49%
H^2 (total variability / sampling variability):  1.32

Test for Heterogeneity:
Q(df = 65) = 91.1555, p-val = 0.0179

Model Results:

estimate      se     zval    pval    ci.lb    ci.ub 
 -5.9373  1.7320  -3.4280  0.0006  -9.3320  -2.5426  *** 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Looking at the effect of body weight at 100 days on both diets on longevity adjusting for strain effect

In [9]:
out4 <- summarizeByCovariate(agingbxd,"AgeAtDeath","Weight02")
summary(out4$outMeta)

`summarise()` ungrouping output (override with `.groups` argument)

Warning message in rma.uni(yi = strainByCov$estimate, sei = strainByCov$std.error):
“Studies with NAs omitted from model fitting.”



 --- ANOVA --- 
 Weight02 Residuals 
      5.4      94.6 

 --- LM --- 
# A tibble: 2 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)   795.         16.7       47.7   1.95e-284
2 Weight02       -3.93        0.463     -8.49  5.74e- 17

 --- LMM --- 
             Estimate Std. Error   t value
(Intercept) 796.33768 19.4085791 41.030190
Weight02     -3.96807  0.4570734 -8.681472

 --- Meta --- 
          estimate        se     zval         pval     ci.lb     ci.ub
Weight02 -4.419632 0.6226359 -7.09826 1.263371e-12 -5.639976 -3.199288



Random-Effects Model (k = 69; tau^2 estimator: REML)

   logLik   deviance        AIC        BIC       AICc 
-228.9662   457.9325   461.9325   466.3715   462.1171   

tau^2 (estimated amount of total heterogeneity): 9.7758 (SE = 4.1470)
tau (square root of estimated tau^2 value):      3.1266
I^2 (total heterogeneity / total variability):   41.90%
H^2 (total variability / sampling variability):  1.72

Test for Heterogeneity:
Q(df = 68) = 134.6758, p-val < .0001

Model Results:

estimate      se     zval    pval    ci.lb    ci.ub 
 -4.4196  0.6226  -7.0983  <.0001  -5.6400  -3.1993  *** 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Looking at the effect of body weight gain on longevity at 100 days on HFD 

In [10]:
hfd <- subset(agingbxd, (agingbxd)$DietCode=='HF')
out5 <- summarizeByCovariate(hfd,"AgeAtDeath","WeightGainPost100days",10)
summary(out5$outLM)

`summarise()` ungrouping output (override with `.groups` argument)




 --- ANOVA --- 
WeightGainPost100days             Residuals 
                  0.7                  99.3 

 --- LM --- 
# A tibble: 2 x 5
  coefname              Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>                    <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)             640.         17.8       35.9   7.09e-132
2 WeightGainPost100days    -1.56        0.898     -1.74  8.26e-  2

 --- LMM --- 
                         Estimate Std. Error    t value
(Intercept)           611.4000983 25.4846136 23.9909503
WeightGainPost100days  -0.2287731  0.9667132 -0.2366504

 --- Meta --- 
                        estimate       se       zval      pval     ci.lb
WeightGainPost100days 0.06412785 1.271151 0.05044863 0.9597649 -2.427283
                         ci.ub
WeightGainPost100days 2.555539



Call:
lm(formula = formula(paste(pheno, "~", cov)), data = newdata)

Residuals:
    Min      1Q  Median      3Q     Max 
-422.33 -104.11    9.66  104.38  457.80 

Coefficients:
                      Estimate Std. Error t value Pr(>|t|)    
(Intercept)           639.9520    17.8428   35.87   <2e-16 ***
WeightGainPost100days  -1.5625     0.8981   -1.74   0.0826 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 156.4 on 434 degrees of freedom
  (13 observations deleted due to missingness)
Multiple R-squared:  0.006926,	Adjusted R-squared:  0.004638 
F-statistic: 3.027 on 1 and 434 DF,  p-value: 0.0826


## Looking at the effect of body weight on longevity at 400 days on CD 

In [11]:
cd <- subset(agingbxd, (agingbxd)$DietCode=='CD')
out6 <- summarizeByCovariate(cd,"AgeAtDeath","Weight06", 10)
summary(out6$outMeta)

`summarise()` ungrouping output (override with `.groups` argument)

Warning message in rma.uni(yi = strainByCov$estimate, sei = strainByCov$std.error):
“Studies with NAs omitted from model fitting.”



 --- ANOVA --- 
 Weight06 Residuals 
      3.1      96.9 

 --- LM --- 
# A tibble: 2 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)   846.         31.0       27.3    2.67e-73
2 Weight06       -2.64        0.978     -2.70   7.55e- 3

 --- LMM --- 
              Estimate Std. Error   t value
(Intercept) 824.810122  35.860127 23.000759
Weight06     -2.022995   1.104843 -1.831024

 --- Meta --- 
         estimate       se       zval      pval     ci.lb    ci.ub
Weight06 0.036554 1.703265 0.02146113 0.9828778 -3.301784 3.374892



Random-Effects Model (k = 24; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc 
-86.2925  172.5850  176.5850  178.8559  177.1850   

tau^2 (estimated amount of total heterogeneity): 12.4293 (SE = 16.3997)
tau (square root of estimated tau^2 value):      3.5255
I^2 (total heterogeneity / total variability):   22.70%
H^2 (total variability / sampling variability):  1.29

Test for Heterogeneity:
Q(df = 23) = 33.4536, p-val = 0.0735

Model Results:

estimate      se    zval    pval    ci.lb   ci.ub 
  0.0366  1.7033  0.0215  0.9829  -3.3018  3.3749    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Looking at the effect of body weight on longevity at 400 days on HFD 

In [12]:
hfd <- subset(agingbxd, (agingbxd)$DietCode=='HF')
out7 <- summarizeByCovariate(hfd,"AgeAtDeath","Weight06", 11)
summary(out7$outMeta)

`summarise()` ungrouping output (override with `.groups` argument)

Warning message in rma.uni(yi = strainByCov$estimate, sei = strainByCov$std.error):
“Studies with NAs omitted from model fitting.”



 --- ANOVA --- 
 Weight06 Residuals 
      1.1      98.9 

 --- LM --- 
# A tibble: 2 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)  736.          27.8       26.5    4.33e-75
2 Weight06      -0.828        0.498     -1.66   9.77e- 2

 --- LMM --- 
               Estimate Std. Error    t value
(Intercept) 707.2942440 31.4338502 22.5010375
Weight06     -0.4456813  0.5187318 -0.8591747

 --- Meta --- 
           estimate        se      zval      pval     ci.lb   ci.ub
Weight06 -0.2216731 0.4586988 -0.483265 0.6289076 -1.120706 0.67736



Random-Effects Model (k = 24; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc 
-59.2258  118.4516  122.4516  124.7226  123.0516   

tau^2 (estimated amount of total heterogeneity): 0.0000 (SE = 1.0443)
tau (square root of estimated tau^2 value):      0.0012
I^2 (total heterogeneity / total variability):   0.00%
H^2 (total variability / sampling variability):  1.00

Test for Heterogeneity:
Q(df = 23) = 25.8725, p-val = 0.3069

Model Results:

estimate      se     zval    pval    ci.lb   ci.ub 
 -0.2217  0.4587  -0.4833  0.6289  -1.1207  0.6774    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Looking at the effect of body weight gain on longevity at 400 days on HFD

In [13]:
hfd <- subset(agingbxd, (agingbxd)$DietCode=='HF')
out8 <- summarizeByCovariate(hfd,"AgeAtDeath","WeightGainPost400days",11)
summary(out8$outMeta)

`summarise()` ungrouping output (override with `.groups` argument)

Warning message in rma.uni(yi = strainByCov$estimate, sei = strainByCov$std.error):
“Studies with NAs omitted from model fitting.”



 --- ANOVA --- 
WeightGainPost400days             Residuals 
                  0.5                  99.5 

 --- LM --- 
# A tibble: 2 x 5
  coefname              Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>                    <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)            709.          16.7       42.5   3.51e-117
2 WeightGainPost400days   -0.547        0.481     -1.14  2.56e-  1

 --- LMM --- 
                         Estimate Std. Error   t value
(Intercept)           701.0214606 21.0762834 33.261152
WeightGainPost400days  -0.5583583  0.4890035 -1.141829

 --- Meta --- 
                        estimate        se      zval      pval     ci.lb
WeightGainPost400days -0.4993763 0.4400145 -1.134909 0.2564135 -1.361789
                          ci.ub
WeightGainPost400days 0.3630362



Random-Effects Model (k = 24; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc 
-56.6331  113.2661  117.2661  119.5371  117.8661   

tau^2 (estimated amount of total heterogeneity): 0.0000 (SE = 1.0143)
tau (square root of estimated tau^2 value):      0.0014
I^2 (total heterogeneity / total variability):   0.00%
H^2 (total variability / sampling variability):  1.00

Test for Heterogeneity:
Q(df = 23) = 25.0034, p-val = 0.3501

Model Results:

estimate      se     zval    pval    ci.lb   ci.ub 
 -0.4994  0.4400  -1.1349  0.2564  -1.3618  0.3630    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Looking at the effect of diet on body weight at 400 days adjusting for strain effect

In [14]:
out9 <- summarizeByDiet(agingbxd,"Weight06")
summary(out9$outMeta)

`summarise()` regrouping output by 'DietCode' (override with `.groups` argument)

`summarise()` regrouping output by 'DietCode' (override with `.groups` argument)

Warning message in rma.uni(measure = "MD", m1i = mean_HF, m2i = mean_CD, sd1i = sd_HF, :
“Studies with NAs omitted from model fitting.”



 --- ANOVA --- 
         DietCode StrainNameCurrent         Residuals 
             45.0              19.1              35.9 

 --- LM --- 
# A tibble: 2 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)     36.6        2.12       17.3  9.99e- 57
2 DietCodeHF      22.4        0.740      30.2  5.87e-133

 --- ANOVAX --- 
                  DietCode          StrainNameCurrent 
                      45.0                       19.1 
DietCode:StrainNameCurrent                  Residuals 
                      10.3                       25.5 

 --- LMX --- 
# A tibble: 2 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)     29.4         2.45      12.0   1.80e-30
2 DietCodeHF      38.6         3.69      10.5   6.38e-24

 --- LMM --- 
            Estimate Std. Error t value
(Intercept) 29.27114  0.8871682 32.9939
DietCodeHF  22.1852


Random-Effects Model (k = 54; tau^2 estimator: REML)

   logLik   deviance        AIC        BIC       AICc 
-201.6873   403.3747   407.3747   411.3153   407.6147   

tau^2 (estimated amount of total heterogeneity): 94.1599 (SE = 22.4653)
tau (square root of estimated tau^2 value):      9.7036
I^2 (total heterogeneity / total variability):   87.35%
H^2 (total variability / sampling variability):  7.91

Test for Heterogeneity:
Q(df = 53) = 456.7811, p-val < .0001

Model Results:

estimate      se     zval    pval    ci.lb    ci.ub 
 22.2509  1.4716  15.1204  <.0001  19.3666  25.1351  *** 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Fat mass and Adiposity in harvested BXD females

In [15]:
BXDagingfat <- "../data/AgingBXDharvest_FatMass.csv"
agingbxdfat <- read.csv(BXDagingfat)
names(agingbxdfat)

[1] "EarTagNumberOriginal"  "EarTagNumberCurrent"   "IDCase"               
 [4] "StrainNameCurrent"     "DietCode"              "Sex"                  
 [7] "DateOfBirth"           "DateOfDeath"           "CauseOfDeath"         
[10] "AgeAtDeath"            "WeightBody"            "WeightSubcutaneousWat"
[13] "WeightPerirenalWat"    "WeightPerigonadalWat"  "Subq.Adiposity"       
[16] "Visceral.Fat.Mass"     "Visceral.Adiposity"    "Total.Fat.Mass"       
[19] "Total.Adiposity"

## Looking at the effect of adiposity in harvested females on longevity adjusting for diet and strain effect

In [16]:
out10 <- summarizeByCovariate2(agingbxdfat,"AgeAtDeath","Total.Adiposity","DietCode", 10)
summary(out10$outMeta)
out11 <- summarizeByCovariate2(agingbxdfat,"AgeAtDeath","Visceral.Adiposity","DietCode", 10)
summary(out11$outMeta)
out12 <- summarizeByCovariate2(agingbxdfat,"AgeAtDeath","Subq.Adiposity","DietCode", 10)
summary(out12$outMeta)

`summarise()` ungrouping output (override with `.groups` argument)

boundary (singular) fit: see ?isSingular




 --- ANOVA --- 
  Total.Adiposity          DietCode StrainNameCurrent         Residuals 
              0.1               1.4               4.8              93.7 

 --- LM --- 
# A tibble: 3 x 5
  coefname        Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>              <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)       459.          45.6      10.1    2.80e-21
2 Total.Adiposity     4.56         1.89      2.41   1.63e- 2
3 DietCodeHF        -74.7         26.6      -2.81   5.28e- 3

 --- LMM --- 
                  Estimate Std. Error   t value
(Intercept)     431.839889  14.789795 29.198505
Total.Adiposity   3.486771   1.638186  2.128434
DietCodeHF      -58.676527  24.427689 -2.402050

 --- Meta --- 
                estimate       se     zval       pval     ci.lb    ci.ub
Total.Adiposity 6.419434 3.088703 2.078359 0.03767628 0.3656878 12.47318



Random-Effects Model (k = 25; tau^2 estimator: REML)

   logLik   deviance        AIC        BIC       AICc 
-100.2262   200.4523   204.4523   206.8084   205.0237   

tau^2 (estimated amount of total heterogeneity): 119.2429 (SE = 65.2982)
tau (square root of estimated tau^2 value):      10.9198
I^2 (total heterogeneity / total variability):   59.21%
H^2 (total variability / sampling variability):  2.45

Test for Heterogeneity:
Q(df = 24) = 55.1097, p-val = 0.0003

Model Results:

estimate      se    zval    pval   ci.lb    ci.ub 
  6.4194  3.0887  2.0784  0.0377  0.3657  12.4732  * 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


`summarise()` ungrouping output (override with `.groups` argument)

boundary (singular) fit: see ?isSingular




 --- ANOVA --- 
Visceral.Adiposity           DietCode  StrainNameCurrent          Residuals 
               0.0                0.8                4.9               94.3 

 --- LM --- 
# A tibble: 3 x 5
  coefname           Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>                 <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)          481.          44.6      10.8    7.12e-24
2 Visceral.Adiposity     3.05         2.14      1.42   1.55e- 1
3 DietCodeHF           -54.8         25.2      -2.17   3.04e- 2

 --- LMM --- 
                    Estimate Std. Error   t value
(Intercept)        442.37047  14.425398 30.666084
Visceral.Adiposity   2.12952   1.881109  1.132055
DietCodeHF         -42.67718  23.337439 -1.828700

 --- Meta --- 
                   estimate       se     zval       pval      ci.lb    ci.ub
Visceral.Adiposity 5.827002 3.445333 1.691274 0.09078447 -0.9257261 12.57973



Random-Effects Model (k = 25; tau^2 estimator: REML)

   logLik   deviance        AIC        BIC       AICc 
-103.0649   206.1299   210.1299   212.4860   210.7013   

tau^2 (estimated amount of total heterogeneity): 141.7205 (SE = 80.9101)
tau (square root of estimated tau^2 value):      11.9046
I^2 (total heterogeneity / total variability):   53.97%
H^2 (total variability / sampling variability):  2.17

Test for Heterogeneity:
Q(df = 24) = 49.6920, p-val = 0.0015

Model Results:

estimate      se    zval    pval    ci.lb    ci.ub 
  5.8270  3.4453  1.6913  0.0908  -0.9257  12.5797  . 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


`summarise()` ungrouping output (override with `.groups` argument)

boundary (singular) fit: see ?isSingular




 --- ANOVA --- 
   Subq.Adiposity          DietCode StrainNameCurrent         Residuals 
              0.7               1.7               4.7              92.9 

 --- LM --- 
# A tibble: 3 x 5
  coefname       Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>             <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)       458.         45.1      10.2    1.20e-21
2 Subq.Adiposity     13.6         4.86      2.79   5.53e- 3
3 DietCodeHF        -60.2        21.3      -2.83   4.93e- 3

 --- LMM --- 
                Estimate Std. Error   t value
(Intercept)    437.87264  13.385179 32.713246
Subq.Adiposity  12.63332   4.472387  2.824738
DietCodeHF     -54.77498  20.731302 -2.642139

 --- Meta --- 
               estimate       se      zval      pval    ci.lb    ci.ub
Subq.Adiposity 3.958852 10.57987 0.3741873 0.7082649 -16.7773 24.69501



Random-Effects Model (k = 25; tau^2 estimator: REML)

   logLik   deviance        AIC        BIC       AICc 
-131.9817   263.9634   267.9634   270.3195   268.5348   

tau^2 (estimated amount of total heterogeneity): 952.5662 (SE = 661.2892)
tau (square root of estimated tau^2 value):      30.8637
I^2 (total heterogeneity / total variability):   58.42%
H^2 (total variability / sampling variability):  2.41

Test for Heterogeneity:
Q(df = 24) = 39.2043, p-val = 0.0260

Model Results:

estimate       se    zval    pval     ci.lb    ci.ub 
  3.9589  10.5799  0.3742  0.7083  -16.7773  24.6950    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Blood metabolites in harvested BXD females

In [17]:
BXDagingmet <- "../data/AgingBXD_harvestedfemales_BloodMetabolites_SR.csv"
agingbxdmetab <- read.csv(BXDagingmet)
names(agingbxdmetab)

[1] "EarTagNumberOriginal" "EarTagNumberCurrent"  "StrainNameCurrent"   
 [4] "DietCode"             "Sex"                  "AgeAtDeath"          
 [7] "WeightBody"           "Glucose"              "TotalChol"           
[10] "Triglycerides"        "FFA"

## Looking at the effect of blood metabolites in harvested females on longevity adjusting for diet and strain effect

In [18]:
out13 <- summarizeByCovariate2(agingbxdmetab,"AgeAtDeath","Glucose","DietCode", 10)
summary(out13$outMeta)
out14 <- summarizeByCovariate2(agingbxdmetab,"AgeAtDeath","TotalChol","DietCode", 10)
summary(out14$outMeta)
out15 <- summarizeByCovariate2(agingbxdmetab,"AgeAtDeath","Triglycerides","DietCode", 10)
summary(out15$outMeta)
out16 <- summarizeByCovariate2(agingbxdmetab,"AgeAtDeath","FFA","DietCode", 10)
summary(out16$outMeta)

`summarise()` ungrouping output (override with `.groups` argument)




 --- ANOVA --- 
          Glucose          DietCode StrainNameCurrent         Residuals 
             10.8               0.0              10.7              78.4 

 --- LM --- 
# A tibble: 3 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)   665.          51.5     12.9     1.88e-30
2 Glucose       -17.8          2.90    -6.14    2.76e- 9
3 DietCodeHF     -3.17        18.8     -0.169   8.66e- 1

 --- LMM --- 
              Estimate Std. Error    t value
(Intercept) 599.314105  33.180205 18.0623989
Glucose     -17.018424   2.780295 -6.1210857
DietCodeHF   -4.328789  18.564941 -0.2331701

 --- Meta --- 
        estimate       se      zval         pval     ci.lb     ci.ub
Glucose -19.4536 3.179116 -6.119187 9.405374e-10 -25.68456 -13.22265



Random-Effects Model (k = 20; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc 
-77.4679  154.9358  158.9358  160.8247  159.6858   

tau^2 (estimated amount of total heterogeneity): 40.5787 (SE = 58.6857)
tau (square root of estimated tau^2 value):      6.3701
I^2 (total heterogeneity / total variability):   21.28%
H^2 (total variability / sampling variability):  1.27

Test for Heterogeneity:
Q(df = 19) = 20.7254, p-val = 0.3522

Model Results:

estimate      se     zval    pval     ci.lb     ci.ub 
-19.4536  3.1791  -6.1192  <.0001  -25.6846  -13.2227  *** 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


`summarise()` ungrouping output (override with `.groups` argument)




 --- ANOVA --- 
        TotalChol          DietCode StrainNameCurrent         Residuals 
              5.2               0.0               8.9              85.9 

 --- LM --- 
# A tibble: 3 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)   545.           48.8    11.2     2.92e-24
2 TotalChol     -38.0          12.6    -3.03    2.67e- 3
3 DietCodeHF      2.70         21.4     0.126   9.00e- 1

 --- LMM --- 
              Estimate Std. Error    t value
(Intercept) 494.882650   25.61165 19.3225585
TotalChol   -42.097510   11.77165 -3.5761772
DietCodeHF    5.590896   20.83364  0.2683591

 --- Meta --- 
           estimate       se      zval        pval     ci.lb     ci.ub
TotalChol -56.94514 21.35688 -2.666361 0.007667736 -98.80385 -15.08643


...


 --- ANOVA --- 
    Triglycerides          DietCode StrainNameCurrent         Residuals 
              3.7               0.3              10.0              86.0 

 --- LM --- 
# A tibble: 3 x 5
  coefname      Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>            <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)      517.          45.3     11.4    3.56e-25
2 Triglycerides    -81.6         27.3     -2.99   3.03e- 3
3 DietCodeHF       -21.5         19.4     -1.11   2.69e- 1

 --- LMM --- 
               Estimate Std. Error   t value
(Intercept)   461.73705   20.24662 22.805631
Triglycerides -84.66676   26.21683 -3.229481
DietCodeHF    -19.74008   19.13961 -1.031373

 --- Meta --- 
               estimate       se      zval       pval     ci.lb    ci.ub
Triglycerides -84.86464 47.25792 -1.795776 0.07253012 -177.4885 7.759172



Random-Effects Model (k = 20; tau^2 estimator: REML)

   logLik   deviance        AIC        BIC       AICc 
-127.9639   255.9279   259.9279   261.8167   260.6779   

tau^2 (estimated amount of total heterogeneity): 24685.0766 (SE = 14058.3851)
tau (square root of estimated tau^2 value):      157.1149
I^2 (total heterogeneity / total variability):   60.61%
H^2 (total variability / sampling variability):  2.54

Test for Heterogeneity:
Q(df = 19) = 53.0842, p-val < .0001

Model Results:

estimate       se     zval    pval      ci.lb   ci.ub 
-84.8646  47.2579  -1.7958  0.0725  -177.4885  7.7592  . 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


`summarise()` ungrouping output (override with `.groups` argument)




 --- ANOVA --- 
              FFA          DietCode StrainNameCurrent         Residuals 
              1.9               0.3              10.6              87.2 

 --- LM --- 
# A tibble: 3 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)    504.          48.3     10.4    9.08e-22
2 FFA            -76.7         32.7     -2.34   1.99e- 2
3 DietCodeHF     -22.9         19.7     -1.16   2.46e- 1

 --- LMM --- 
             Estimate Std. Error   t value
(Intercept) 467.73642   24.60890 19.006795
FFA         -76.28935   31.62933 -2.411981
DietCodeHF  -20.91444   19.52872 -1.070958

 --- Meta --- 
     estimate       se      zval       pval     ci.lb    ci.ub
FFA -99.36553 58.00361 -1.713092 0.08669557 -213.0505 14.31945



Random-Effects Model (k = 20; tau^2 estimator: REML)

   logLik   deviance        AIC        BIC       AICc 
-132.2759   264.5518   268.5518   270.4406   269.3018   

tau^2 (estimated amount of total heterogeneity): 36376.4437 (SE = 20973.4428)
tau (square root of estimated tau^2 value):      190.7261
I^2 (total heterogeneity / total variability):   62.45%
H^2 (total variability / sampling variability):  2.66

Test for Heterogeneity:
Q(df = 19) = 54.4242, p-val < .0001

Model Results:

estimate       se     zval    pval      ci.lb    ci.ub 
-99.3655  58.0036  -1.7131  0.0867  -213.0505  14.3195  . 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Looking at the effect of blood metabolites in harvested females on body weight adjusting for diet and strain effect

In [19]:
out13a <- summarizeByCovariate2(agingbxdmetab,"WeightBody","Glucose","DietCode", 10)
summary(out13a$outMeta)
out14a <- summarizeByCovariate2(agingbxdmetab,"WeightBody","TotalChol","DietCode", 10)
summary(out14a$outMeta)
out15a <- summarizeByCovariate2(agingbxdmetab,"WeightBody","Triglycerides","DietCode", 10)
summary(out15a$outMeta)
out16a <- summarizeByCovariate2(agingbxdmetab,"WeightBody","FFA","DietCode", 10)
summary(out16a$outMeta)

`summarise()` ungrouping output (override with `.groups` argument)




 --- ANOVA --- 
          Glucose          DietCode StrainNameCurrent         Residuals 
             11.2              32.1              14.4              42.3 

 --- LM --- 
# A tibble: 3 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)   30.9          3.29       9.39   1.85e-18
2 Glucose        0.656        0.186      3.54   4.69e- 4
3 DietCodeHF    18.6          1.20      15.5    1.22e-39

 --- LMM --- 
              Estimate Std. Error   t value
(Intercept) 20.5168026   2.391008  8.580818
Glucose      0.7466654   0.182183  4.098435
DietCodeHF  18.3094668   1.196126 15.307305

 --- Meta --- 
         estimate        se    zval       pval      ci.lb     ci.ub
Glucose 0.4598036 0.2130555 2.15814 0.03091696 0.04222245 0.8773848



Random-Effects Model (k = 20; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc 
-25.2450   50.4900   54.4900   56.3789   55.2400   

tau^2 (estimated amount of total heterogeneity): 0.3702 (SE = 0.2658)
tau (square root of estimated tau^2 value):      0.6084
I^2 (total heterogeneity / total variability):   49.09%
H^2 (total variability / sampling variability):  1.96

Test for Heterogeneity:
Q(df = 19) = 37.8627, p-val = 0.0062

Model Results:

estimate      se    zval    pval   ci.lb   ci.ub 
  0.4598  0.2131  2.1581  0.0309  0.0422  0.8774  * 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


`summarise()` ungrouping output (override with `.groups` argument)




 --- ANOVA --- 
        TotalChol          DietCode StrainNameCurrent         Residuals 
             24.5              21.0              13.5              41.0 

 --- LM --- 
# A tibble: 3 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)    31.3         2.93      10.7    1.10e-22
2 TotalChol       3.54        0.755      4.69   4.32e- 6
3 DietCodeHF     16.8         1.29      13.0    1.15e-30

 --- LMM --- 
             Estimate Std. Error   t value
(Intercept) 21.578379  1.9013292 11.349102
TotalChol    3.892304  0.7387224  5.268967
DietCodeHF  16.367047  1.2783525 12.803234

 --- Meta --- 
          estimate      se     zval       pval      ci.lb    ci.ub
TotalChol 2.003196 1.13554 1.764091 0.07771665 -0.2224216 4.228814


...


 --- ANOVA --- 
    Triglycerides          DietCode StrainNameCurrent         Residuals 
              1.7              36.7              18.2              43.4 

 --- LM --- 
# A tibble: 3 x 5
  coefname      Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>            <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)      35.9          2.80     12.8    3.96e-30
2 Triglycerides     3.79         1.68      2.25   2.51e- 2
3 DietCodeHF       19.2          1.20     16.1    5.56e-42

 --- LMM --- 
               Estimate Std. Error   t value
(Intercept)   26.545640   1.784647 14.874445
Triglycerides  3.636805   1.664512  2.184907
DietCodeHF    19.094587   1.192324 16.014595

 --- Meta --- 
              estimate       se     zval       pval     ci.lb    ci.ub
Triglycerides 3.784449 1.532434 2.469567 0.01352768 0.7809326 6.787965



Random-Effects Model (k = 20; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc 
-66.5844  133.1689  137.1689  139.0577  137.9189   

tau^2 (estimated amount of total heterogeneity): 4.2383 (SE = 12.5387)
tau (square root of estimated tau^2 value):      2.0587
I^2 (total heterogeneity / total variability):   9.27%
H^2 (total variability / sampling variability):  1.10

Test for Heterogeneity:
Q(df = 19) = 21.1225, p-val = 0.3301

Model Results:

estimate      se    zval    pval   ci.lb   ci.ub 
  3.7844  1.5324  2.4696  0.0135  0.7809  6.7880  * 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


`summarise()` ungrouping output (override with `.groups` argument)




 --- ANOVA --- 
              FFA          DietCode StrainNameCurrent         Residuals 
              0.1              37.8              17.9              44.2 

 --- LM --- 
# A tibble: 3 x 5
  coefname    Estimate `Std. Error` `t value` `Pr(>|t|)`
  <chr>          <dbl>        <dbl>     <dbl>      <dbl>
1 (Intercept)   37.1           2.99    12.4     2.20e-28
2 FFA            0.477         2.03     0.235   8.15e- 1
3 DietCodeHF    19.5           1.23    15.9     5.52e-41

 --- LMM --- 
              Estimate Std. Error    t value
(Intercept) 27.9477793   1.977795 14.1307791
FFA          0.5210708   2.007629  0.2595454
DietCodeHF  19.3643010   1.222808 15.8359338

 --- Meta --- 
    estimate       se     zval       pval     ci.lb    ci.ub
FFA 4.336819 2.091732 2.073315 0.03814298 0.2370998 8.436538



Random-Effects Model (k = 20; tau^2 estimator: REML)

  logLik  deviance       AIC       BIC      AICc 
-71.0615  142.1230  146.1230  148.0118  146.8730   

tau^2 (estimated amount of total heterogeneity): 15.8173 (SE = 24.8591)
tau (square root of estimated tau^2 value):      3.9771
I^2 (total heterogeneity / total variability):   19.19%
H^2 (total variability / sampling variability):  1.24

Test for Heterogeneity:
Q(df = 19) = 22.4464, p-val = 0.2626

Model Results:

estimate      se    zval    pval   ci.lb   ci.ub 
  4.3368  2.0917  2.0733  0.0381  0.2371  8.4365  * 

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Coefficient of variation in longevity

In [20]:
longevityMeanSD <- group_by(agingbxd,DietCode,StrainNameCurrent) %>% 
             summarise(n=n(),mean=mean(AgeAtDeath),sd=sd(AgeAtDeath) ) %>% 
             pivot_wider(names_from = DietCode,values_from = c(n,mean,sd)) %>%
             filter((n_CD>=6) & (n_HF>=6))

`summarise()` regrouping output by 'DietCode' (override with `.groups` argument)



In [21]:
rma.uni(n1i=n_CD,n2i=n_HF,m1i=mean_CD,m2i=mean_HF,sd1i=sd_CD,sd2i=sd_HF,
        measure="CVR",data=longevityMeanSD)


Random-Effects Model (k = 50; tau^2 estimator: REML)

tau^2 (estimated amount of total heterogeneity): 0.0996 (SE = 0.0424)
tau (square root of estimated tau^2 value):      0.3155
I^2 (total heterogeneity / total variability):   48.22%
H^2 (total variability / sampling variability):  1.93

Test for Heterogeneity:
Q(df = 49) = 94.9778, p-val < .0001

Model Results:

estimate      se    zval    pval    ci.lb   ci.ub 
  0.0757  0.0653  1.1591  0.2464  -0.0523  0.2037    

---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


## Session info

In [22]:
sessionInfo()

R version 4.0.2 (2020-06-22)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS Catalina 10.15.7

Matrix products: default
BLAS:   /Library/Frameworks/R.framework/Versions/4.0/Resources/lib/libRblas.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/4.0/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] metafor_2.4-0   lme4_1.1-26     Matrix_1.2-18   vioplot_0.3.5  
 [5] zoo_1.8-8       sm_2.2-5.6      forcats_0.5.0   stringr_1.4.0  
 [9] dplyr_1.0.1     purrr_0.3.4     readr_1.3.1     tidyr_1.1.1    
[13] tibble_3.0.3    ggplot2_3.3.2   tidyverse_1.3.0

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.6       lubridate_1.7.9  lattice_0.20-41  ps_1.3.4        
 [5] utf8_1.1.4       assertthat_0.2.1 digest_0.6.25    IRdisplay_0.7.0 
 [9] R6_2.5.0         cellranger_1.1.0 repr_1.1.